# Predicting Dynamic Memory Requirements for Scientific Workflow Tasks

### Setup BASE_DIR
- Path to tsb_resource_allocation_data
- No / at the end

In [8]:
BASE_DIR = 'C:/privat/Bachelor_Work/pytonProject/k-segments-traces-main/k-segments-traces-main' 

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import pandas as pd
from tsb_resource_allocation.witt_task_model import WittTaskModel
from tsb_resource_allocation.tovar_task_model import TovarTaskModel
from tsb_resource_allocation.simulation import Simulation
from tsb_resource_allocation.k_segments_model import KSegmentsModel
from tsb_resource_allocation.file_events_model import FileEventsModel
from tsb_resource_allocation.default_model import DefaultModel
sns.set_theme(style="darkgrid")

# Helper methods

def get_file_names(directory, number_of_files = -1):
    file_names = [name.rsplit('_',1)[0] for name in os.listdir(directory) if not os.path.isdir(f"{directory}{name}") and name.endswith("_memory.csv")]
    if number_of_files != -1:
        return file_names[:number_of_files]
    return file_names

def run_simulation(directory, training, test, monotonically_increasing = True, k = 4, collection_interval = 2):
    
    # MODELS
    simulations = []
    
    # KSegments retry: selective
    task_model = KSegmentsModel(k = k, monotonically_increasing = monotonically_increasing)
    simulation = Simulation(task_model, directory, retry_mode = 'selective', provided_file_names = training)
    simulations.append(simulation)
    
    # KSegments retry: selective - NO UNDERPREDICTION
    task_model = KSegmentsModel(k = k, monotonically_increasing = monotonically_increasing, time_mode = -1)
    simulation = Simulation(task_model, directory, retry_mode = 'selective', provided_file_names = training)
    #simulations.append(simulation)
    
    # KSegments retry: partial
    task_model = KSegmentsModel(k = k, monotonically_increasing = monotonically_increasing)
    simulation = Simulation(task_model, directory, retry_mode = 'partial', provided_file_names = training)
    simulations.append(simulation)
    
    # WITT LR MEAN+- TASK MODEL 
    task_model = WittTaskModel(mode = "mean+-")
    simulation = Simulation(task_model, directory, retry_mode = 'full', provided_file_names = training)
    simulations.append(simulation)

    # TOVAR TASK MODEL - full retry
    task_model = TovarTaskModel()
    simulation = Simulation(task_model, directory, retry_mode = 'full', provided_file_names = training)
    simulations.append(simulation)
    
     # TOVAR TASK MODEL - tovar retry
    task_model = TovarTaskModel()
    simulation = Simulation(task_model, directory, retry_mode = 'tovar', provided_file_names = training)
    simulations.append(simulation)
    
    # Default Model
    task_model = DefaultModel()
    simulation = Simulation(task_model, directory, retry_mode = 'full', provided_file_names = training)
    simulations.append(simulation)
    
    waste, retries, runtimes = [0 for _ in range(len(simulations))],[0 for _ in range(len(simulations))],[0 for _ in range(len(simulations))]
    for file_name in test:
        for i,s in enumerate(simulations):
            result = s.execute(file_name, True)
            waste[i] += ((result[0]/1000) * collection_interval)
            retries[i] += result[1]
            runtimes[i] += (result[2] * collection_interval)
    
    avg_waste = list(map(lambda w: w / len(test), waste))
    avg_retries = list(map(lambda r: r / len(test), retries))
    avg_runtime = list(map(lambda r: r / len(test), runtimes))
    
    return avg_waste, avg_retries, avg_runtime

# Table Generation

In [11]:
import random
# OUTPUT = ( [Waste: [Witt: 25, Tovar: 25, k-segments:25], [50] , [75]], [Retries], [Runtime])
def benchmark_task(task_dir = f'{BASE_DIR}/eager/markduplicates'):
    directory = task_dir
    file_names_orig = get_file_names(directory)

    percentages = [0.25, 0.5, 0.75]

    x = []
    y_waste = []
    y_retries = []
    y_runtime = []

    file_names = list(filter(lambda x: len(pd.read_csv(f'{directory}/{x}_memory.csv', skiprows = 3)) >= 8, file_names_orig))
    if len(file_names) == 0:
        return -1
    print(f'Usable Data: {len(file_names)}/{len(file_names_orig)}')
    
    for i in [ int(len(file_names)*p) for p in percentages ]:
        training = file_names[:i]
        test = file_names[i:] # file_names[i:] - other mode
        print(f"training: {len(training)}, test: {len(test)}",end="\r", flush=True)
        avg_waste, avg_retries, avg_runtime = run_simulation(directory, training, test, k = 4)
        x.append(i)
        y_waste.append(list(map(lambda w: round(w, 2),avg_waste)))
        y_retries.append(avg_retries)
        y_runtime.append(avg_runtime)

    return (y_waste, y_retries, y_runtime)

# Test folder

In [14]:
base_directory = f'{BASE_DIR}/sarek'
workflow_tasks = [os.path.join(base_directory, item) for item in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, item))]
workflow_tasks = [task for task in workflow_tasks if len(os.listdir(task)) > 40]

categories = ["Wastage", "Retries", "Runtime"]
percentages = ["25%", "50%", "75%"]

# 0 = WASTE, 1 = RETRIES, 2 = RUNTIME
for task in workflow_tasks:
    r = benchmark_task(task)
    if r == -1:
        continue
    task_name = os.path.basename(task)
    m = ', '.join(map(str, r[0][2]))
    print(f'{task_name}')
    for i, category in enumerate(categories): 
        for j, percentage in enumerate(percentages): 
            print(f'{category} {percentage}: {r[i][j]}')

Usable Data: 432/432
BWAMEM1_MEM24, test: 108
Wastage 25%: [115.72, 115.72, 546.85, 1067.02, 6346.57, 2466.86]
Wastage 50%: [124.16, 124.16, 390.21, 396.07, 1170.3, 2497.02]
Wastage 75%: [121.97, 121.97, 472.22, 290.09, 290.09, 2465.13]
Retries 25%: [0.0, 0.0, 0.1697530864197531, 0.41358024691358025, 0.41358024691358025, 0.0]
Retries 50%: [0.0, 0.0, 0.08333333333333333, 0.05555555555555555, 0.05555555555555555, 0.0]
Retries 75%: [0.0, 0.0, 0.12962962962962962, 0.0, 0.0, 0.0]
Runtime 25%: [110.58641975308642, 110.58641975308642, 127.75308641975309, 152.90740740740742, 152.90740740740742, 110.58641975308642]
Runtime 50%: [111.91666666666667, 111.91666666666667, 120.70370370370371, 118.56481481481481, 118.56481481481481, 111.91666666666667]
Runtime 75%: [110.29629629629629, 110.29629629629629, 123.92592592592592, 110.29629629629629, 110.29629629629629, 110.29629629629629]
Usable Data: 36/36
CRAM_QC_MOSDEPTH_SAMTOOLS_MOSDEPTH
Wastage 25%: [211.33, 171.75, 72.11, 292.7, 4150.7, 88.2]
Wastag